# Data Preprocessing for the Bachelorette Predictor
### Kwame V. Taylor

I need to preprocess the data from ```joined_df.csv``` to ready it for exploration and modeling. Then I will move on to continue my code in my main notebook, ```bachelorette-predictor.ipynb``` to re-explore the new joined DataFrame, as well as begin modeling.

### Set up Env

### Import the tidied data

### Get dummies

### Create target feature (to predict on)